This workflow follows the framework of two tutorials on lidar [pre-processing](https://rapidlasso.com/2013/10/13/tutorial-lidar-preparation/) and [information extraction](https://rapidlasso.com/2013/10/20/tutorial-derivative-production/) published by Martin Isenburg. It assumes that your lidar data is in tiles and has ground returns already classified. 

0. lasindex
1. lasheight
2. lasclassify
3. lasboundary
4. pit-free CHM

In [1]:
import os
import shutil
from pyFIRS.utils import lastools
import glob
import geopandas as gpd, pandas as pd

In [2]:
las = lastools.useLAStools('C:/LAStools/bin')

## Specify some key parameters for the processing pipeline

In [3]:
# where the raw lidar data is currently stored
raw_tiles = 'C:/Swinomish_Lidar/data/source/2016_Lidar/points/*.laz'
workdir = os.path.abspath('C:/Swinomish_Lidar/lidar_processing')

num_cores=4 # how many cores to use for parallel processing

Take a look at the format of the lidar data provided by the vendor.

In [4]:
vendor_tiles = glob.glob(raw_tiles)
las.lasinfo(i=vendor_tiles[0], echo=True);


lasinfo (180812) report for 'C:/Swinomish_Lidar/data/source/2016_Lidar/points\dropped.laz'
reporting all LAS header entries:
  file signature:             'LASF'
  file source ID:             0
  global_encoding:            1
  project ID GUID data 1-4:   00000000-0000-0000-6557-747300004157
  version major.minor:        1.2
  system identifier:          'LAStools (c) by rapidlasso GmbH'
  generating software:        'las2las (version 180812)'
  file creation day/year:     88/2017
  header size:                227
  offset to point data:       2392
  number var. length records: 3
  point data format:          1
  point data record length:   33
  number of point records:    104870
  number of points by return: 103887 977 6 0 0
  scale factor x y z:         0.01 0.01 0.01
  offset x y z:               0 0 0
  min x y z:                  1145300.79 1115332.82 3.61
  max x y z:                  1147246.39 1115803.65 37.63
variable length header record 1 of 3:
  reserved             0
  us

## Set up the workspace 

In [5]:
# define data handling directories
raw, interim, processed = os.path.join(workdir,'raw'), os.path.join(workdir,'interim'), os.path.join(workdir,'processed')

## 1. Get the raw data into our working directory

In [6]:
%%time
# move the tiles over to our working directory
las.las2las(i=raw_tiles,
            odir=raw,
            drop_withheld=True, # drop any points flagged as withheld by vendor
            drop_class=(7,18), # drop any points classified as noise by vendor
            olaz=True,
            cores=num_cores)
print('Done moving tiles into working directory.')

Done moving tiles into working directory.
Wall time: 5min 31s


In [7]:
%%time
# create spatial indexes for the input files
infiles = os.path.join(raw,'*.laz')

las.lasindex(i=infiles, 
             cores=num_cores)

print("Done adding spatial indexes.")

Done adding spatial indexes.
Wall time: 3min 17s


In [17]:
%%time
# retile to incorporate buffers around tiles
infiles = os.path.join(raw, '*.laz')
odir = os.path.join(interim, 'retiled')

las.lastile(i=infiles,
            buffer=100,
            flag_as_withheld=True,
            olaz=True,
            odir=odir,
            cores=num_cores)


removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1133000_1113000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1133000_1114000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1133000_1115000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1133000_1116000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1134000_1113000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1135000_1113000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1134000_1114000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1135000_1114000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1136000_1113000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1137000_1113000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1136000_1114000.laz
removing C:\Swinomish_Lidar\lidar_processing\interim\retiled\1137000_1114000.laz
removing C:\Swinomish_Lidar

## 2. Classify points in the lidar point cloud
Remove noise and identify high vegetation and buildings.

In [18]:
%%time
# remove noise
infiles=os.path.join(interim, 'retiled', '*.laz')
odir = os.path.join(interim, 'denoised')

las.lasnoise(i=infiles, 
             remove_noise=True,
             odir=odir, 
             olaz=True, 
             cores=num_cores) # use parallel processing

print('Done denoising tiles.')

Done denoising tiles.
Wall time: 13min 6s


In [19]:
%%time
# calculate height aboveground
infiles=os.path.join(interim, 'denoised', '*.laz')
odir = os.path.join(interim, 'lasheight')

las.lasheight(i=infiles,
              odir=odir, 
              olaz=True, 
              cores=num_cores) # use parallel processing

print('Done calculating height above ground.')

Done calculating height above ground.
Wall time: 23min 56s


In [21]:
%%time
# classify some unclassified points into buildings and high vegetation
infiles = os.path.join(interim, 'lasheight', '*.laz')
odir = os.path.join(interim, 'classified')

las.lasclassify(i=infiles,
                odir=odir,
                olaz=True,
                ignore_class=(2,9,10,11,13,14,15,16,17), # ignore points already classified meaningfully
                cores=num_cores) # use parallel processing

print('Done classifying lidar tiles.')

Done classifying lidar tiles.
Wall time: 51min 33s


In [22]:
%%time
# move classified points into processed folder, trimmed to tile buffers
infiles = os.path.join(interim, 'classified', '*.laz')
odir = os.path.join(processed, 'points')

las.lastile(i=infiles,
            odir=odir,
            olaz=True,
            remove_buffer=True, # drop buffers
            set_user_data=0,
            cores=num_cores)

print('Done trimming classified lidar tiles.')

Done trimming classified lidar tiles.
Wall time: 7min 7s


In [23]:
%%time
# produce a shapefile showing layout of tiles
infiles = os.path.join(processed, 'points', '*.laz')
outfile = os.path.join(processed, 'vectors', 'tiles.shp')

las.lasboundary(i=infiles,
            o=outfile,
            oshp=True,
            use_bb=True, # use bounding box of tiles
            overview=True,
            labels=True,
            cores=num_cores) # use parallel processing

print('Produced shapefile showing processed tile boundaries.')

Produced shapefile showing processed tile boundaries.
Wall time: 168 ms


In [24]:
# %%time
# remove intermediate lidar files to conserve storage space
# shutil.rmtree(os.path.join(interim, 'lastiles'))
# shutil.rmtree(os.path.join(interim, 'lasnoise'))
# shutil.rmtree(os.path.join(interim, 'lasheight'))

## 3. Generate a Bare Earth Model
This assumes that there are already ground-classified points

In [41]:
%%time
# generate tiles of the bare earth model
infiles = os.path.join(interim, 'classified', '*.laz')
odir = os.path.join(processed, 'rasters', 'DEM_tiles')

las.las2dem(i=infiles,
            odir=odir,
            obil=True, # create tiles as .bil rasters
            cores=num_cores,
            keep_class=2, # keep ground-classified returns only
            thin_with_grid=1, # use a 1 x 1 resolution for creating the TIN for the DEM
            extra_pass=True, # uses two passes over data to execute DEM creation more efficiently
            use_tile_bb=True) # remove buffers from tiles

print('Done producing bare earth tiles.')

Done producing bare earth tiles.
Wall time: 11min 23s


In [42]:
%%time
# create a merged DEM file
infiles = os.path.join(processed, 'rasters', 'DEM_tiles', '*.bil')
outfile = os.path.join(processed, 'rasters', 'dem.tif')

las.lasgrid(i=infiles,
            merged=True,
            o=outfile)

print('Done producing merged DEM GeoTiff.')

Done producing merged DEM GeoTiff.
Wall time: 2min 26s


Create a hillshade raster.

In [35]:
%%time
# generate hillshade tiles of the bare earth model
infiles = os.path.join(interim, 'classified', '*.laz')
odir = os.path.join(processed, 'rasters', 'hillshade_tiles')

las.las2dem(i=infiles,
            odir=odir,
            obil=True, # create tiles as .bil rasters
            cores=num_cores,
            hillshade=True,
            keep_class=2, # keep ground-classified returns only
            thin_with_grid=1, # use a 0.5 x 0.5 resolution for creating the TIN for the DEM
            extra_pass=True, # uses two passes over data to execute DEM creation more efficiently
            use_tile_bb=True) # remove buffers from tiles

print('Done producing hillshade bare earth tiles.')

Done producing hillshade bare earth tiles.
Wall time: 11min 36s


In [36]:
%%time
# create a merged hillshade raster
infiles = os.path.join(processed, 'rasters', 'hillshade_tiles', '*.bil')
outfile = os.path.join(processed, 'rasters', 'hillshade.tif')

las.lasgrid(i=infiles,
            merged=True,
            o=outfile)

print('Done producing merged hillshade GeoTiff.')

Done producing merged hillshade GeoTiff.
Wall time: 2min 43s


## 4. Generate a shapefile showing locations of buildings

In [37]:
%%time
# generate a shapefile showing building boundaries for each tile
infiles = os.path.join(interim, 'classified', '*.laz')
odir = os.path.join(interim, 'building_tiles')

las.lasboundary(i=infiles,
                odir=odir,
                keep_class=6, # use only building-classified points
                disjoint=True, # compute separate polygons for each building
                concavity=3, # map concave boundary if edge length >= 3ft
                cores=num_cores)

print('Done producing building footprints in buffered tiles.')

Done producing building footprints in buffered tiles.
Wall time: 5min 41s


In [38]:
%%time
# generate shapefiles with the boundaries of each tile
infiles = os.path.join(processed, 'points', '*.laz')
odir = os.path.join(interim, 'tile_boundaries')

las.lasboundary(i=infiles,
                odir=odir,
                oshp=True,
                use_tile_bb=True,
                cores=num_cores)

print('Done producing boundaries of unbuffered tiles.')

Done producing boundaries of unbuffered tiles.
Wall time: 32.6 s


In [39]:
%%time
# clip building tiles to keep only buildings whose centroid falls in the unbuffered tiles
building_tiles = glob.glob(os.path.join(interim, 'building_tiles', '*.shp'))
odir = os.path.join(processed, 'vectors', 'building_tiles')

for poly_shp in building_tiles:
    fname = os.path.basename(poly_shp)
    tile_shp = os.path.join(interim, 'tile_boundaries', fname)
    lastools.clean_tile(poly_shp, tile_shp, odir, simp_tol=1, simp_topol=True)

print('Done producing building footprints in cleaned (unbuffered) tiles.')

Done producing building footprints in cleaned (unbuffered) tiles.
Wall time: 34.9 s


In [40]:
%%time
# merge the cleaned building tiles together
building_tiles = glob.glob(os.path.join(processed, 'vectors', 'building_tiles', '*.shp'))
gdflist = [gpd.read_file(tile) for tile in building_tiles]
merged = gpd.GeoDataFrame(pd.concat(gdflist, ignore_index=True))
merged.crs = gdflist[0].crs
merged.to_file(os.path.join(processed,'vectors','buildings.shp'))

Wall time: 5.08 s
